# Imports:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from configuration.general_config import ITAY_CATMA_CONFIG
# from catma.catma_dataset_creation import CatmaDatasetCreation
from catma.archives_unpacking import CatmaUnpacking
from catma.validations import get_single_validated_file_content
from configuration.general_config import LABEL_REPLACEMENTS_MAPPING, LABELS_TO_DROP, CATMA_XML_ANNOTATION_DIR, \
    AFTER_TEXT_CONTEXT_SIZE, BEFORE_TEXT_CONTEXT_SIZE, START_CHAR_END_CHAR_EXTRACTION_REGEX, COMMITTEE_REGEX, \
    PROTOCOL_NUMBER_REGEX
from configuration.df_columns import LABEL_COLUMN, TEXT_COLUMN, TEXT_SEGMENT_CATMA_ID_COLUMN, START_CHAR_COLUMN, \
    END_CHAR_COLUMN, CLEAN_TEXT_COLUMN, PROTOCOL_NUMBER_COLUMN, AFTER_TEXT_CONTEXT_COLUMN, BEFORE_TEXT_CONTEXT_COLUMN, \
    LABEL_CATMA_ID_COLUMN, FILE_COLUMN, COMMITTEE_COLUMN
from catma.dataset_creation.speaker_level_dataset_creation import SpeakerLevelDatasetCreator

In [3]:
import os
import re
import glob
from bs4 import BeautifulSoup
import pandas as pd
from typing import List, Optional
from dataclasses import dataclass

In [4]:
pd.set_option('display.max_colwidth', 255)

# Extract speakers from text:

### Choose text to start with:

In [5]:
catma_config = ITAY_CATMA_CONFIG
catma_unpacking = CatmaUnpacking(tar_gzs_dir=catma_config.input_catma_tar_gzs_data_path,
                                 unpacked_protocol_archives_dir=catma_config.unpacked_protocol_archives_path)
catma_unpacking.unpack_tar_gz_files()
valid_protocol_dirs = catma_unpacking.get_valid_unpacked_protocol_dirs()

A protocol shouldn't have multiple annotation xml file. Bad protocol dir: /home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_2_17.05.16. We will rename the shorter xml files
A protocol shouldn't have multiple annotation xml file. Bad protocol dir: /home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_26_06-07-2009. We will rename the shorter xml files
A protocol shouldn't have multiple annotation xml file. Bad protocol dir: /home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_3_2006-05-21. We will rename the shorter xml files
A protocol shouldn't have multiple annotation xml file. Bad protocol dir: /home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/ועדת_הכנסת_2006-05-17-01לקידוד. We will rename the shorter xml files
66 out of 87 unpacked tar_gz files are good
Protocol dirs without annotation for instance are: ['/home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_arc

In [47]:
# os.path.basename('/home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_מספר_2_parts')

In [6]:
protocol_dir = [protocol_dir for protocol_dir in valid_protocol_dirs if os.path.basename(protocol_dir) == "פרוטוקול_מספר_42_parts"][0] # # work on "ועדת הכספים חלקי דיון"-> "פרוטוקול מספר 42

In [49]:
protocol_dir

'/home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_מספר_42_parts'

### get_speaker_level_df_from_protocol_dir:

In [316]:
speaker_level_dataset_creator = SpeakerLevelDatasetCreator(remove_new_line_and_tab=ITAY_CATMA_CONFIG.remove_new_line_and_tab)

In [313]:
i = 11

In [317]:
print(i, valid_protocol_dirs[i])
temp_df = speaker_level_dataset_creator.get_df_from_protocol_dir(valid_protocol_dirs[i])
# temp_df = speaker_level_dataset_creator.get_df_from_protocol_dir(protocol_dir)
i = i + 1
# print(temp_df.head(4))
temp_df[~temp_df["label"].isna()].head(3)
# temp_df.sort_values("speaker_text_clean_number_of_words", ascending=False)

12 /home/amit/Desktop/TAU/thesis/itay_catma_data/unpacked_protocol_archives/פרוטוקול_467_ריק
Important: check if you need to remove new line and tab or not (depends on dataset).
            you can change it in the init of CatmaDatasetCreation.
            You can validate it using CATMA analyze with some tags to check.


,speaker_name_start_char,speaker_text_start_char,speaker_end_char,speaker_name,speaker_text,label,label_start_char,label_end_char,file,committee,protocol_number
262,32421,32437,32602,יוסי אדלשטיין:,"אסביר לכבוד היושב-ראש, אנחנו היום, כבוד היושב-ראש, על-פי ההתחייבות שלנו בבג""ץ ובאישור בג""ץ, פועלים היום תוך 10 ימים. אנחנו צריכים לראות את קצב ההסתננות, רבותי.",Judicial decision,32439,32543,פרוטוקול_467_ריק,ועדת הפנים והגנת הסביבה,פרוטוקול מס' 467000999


In [315]:
temp_df.head(10)

,speaker_name_start_char,speaker_text_start_char,speaker_end_char,speaker_name,speaker_text,label,label_start_char,label_end_char
0,1500,1519,2337,"היו""ר יעקב ליצמן:","אני מבקש לפתוח את ישיבת ועדת הכספים. על סדר היום הצעת חוק הסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2006), התשס""ו-2006 – פרק ד': מסים, סעיפים 13-15. למען הפרוטוקול אני רוצה להודיע שבסעיף הנוגע לפנסיה תקצי...",None,None,None
1,2337,2348,3639,ג'קי מצא:,"ברוכים הנמצאים, ובהצלחה לכולם. פרק ד' מתחיל בעמוד 305. אנחנו נמצאים בסעיף 13 – תיקון פקודת מס הכנסה. בפסקה (1) נקבע כי אחרי סעיף 32(14) בפקודת מס הכנסה יבוא סעיף 32(15). פקודת מס הכנסה עושה הבחנה בין הוצאה לשמירה על הקיים ובין הוצאה ל...",judicial decision turns turns,2959,3187
2,2337,2348,3639,ג'קי מצא:,"ברוכים הנמצאים, ובהצלחה לכולם. פרק ד' מתחיל בעמוד 305. אנחנו נמצאים בסעיף 13 – תיקון פקודת מס הכנסה. בפסקה (1) נקבע כי אחרי סעיף 32(14) בפקודת מס הכנסה יבוא סעיף 32(15). פקודת מס הכנסה עושה הבחנה בין הוצאה לשמירה על הקיים ובין הוצאה ל...",None,None,None
3,3639,3651,3674,אלי אפללו:,גם לרכישת מקצוע?,None,None,None
4,3674,3685,4805,ג'קי מצא:,"הוצאות לרכישת מקצוע לא מותרות בניכוי, כי אלה הוצאות שמשמשות לשנים ארוכות מאוד, להבדיל מהוצאות השתלמות אחרות שמותרות בניכוי. בשל העובדה שההוצאות לרכישת לימודים אינן מותרות בניכוי, במסגרת תוכנית המסים הרב-שנתית מיולי 2005 קבענו מתן הקלה במס למי ש...",None,None,None
5,4805,4817,4852,"ישראל כ""ץ:","זאת ההצעה עכשיו, או מה שהיה?",None,None,None
6,4852,4863,5230,ג'קי מצא:,"מתן חצי נקודת זיכוי לרכישת תואר ראשון או תואר שני, לשלוש שנים או שנתיים בהתאמה, נקבע בחקיקה ועבר ביולי 2005. אנחנו עושים כאן הרחבה לטובת אחיות, מורים, רואי חשבון והנדסאים שמסיימים לימודים בהיקף של 1,700 שעות אקדמאיות, אבל סוג הלימודים שלהם, או המוס...",None,None,None
7,5230,5242,5267,"ישראל כ""ץ:",והיכן אתם מורידים?,None,None,None
8,5267,5278,5532,ג'קי מצא:,"אנחנו לא מורידים. אנחנו מבהירים שמקום שמדובר בהוצאות לרכישת לימודים או רכישת מקצוע זאת הוצאה שאינה מותרת בניכוי. זאת רק הבהרה. לאורך השנים זאת היתה עמדת רשות המסים, וגם קיבלה גושפנקה, כפי שאמרתי, בשני פסקי-דין של בתי-משפט מחוזיים שנתפרסמו לאחרונה.",judicial decision turns turns,5408,5528
9,5267,5278,5532,ג'קי מצא:,"אנחנו לא מורידים. אנחנו מבהירים שמקום שמדובר בהוצאות לרכישת לימודים או רכישת מקצוע זאת הוצאה שאינה מותרת בניכוי. זאת רק הבהרה. לאורך השנים זאת היתה עמדת רשות המסים, וגם קיבלה גושפנקה, כפי שאמרתי, בשני פסקי-דין של בתי-משפט מחוזיים שנתפרסמו לאחרונה.",None,None,None


In [283]:
# temp_df = speaker_level_dataset_creator.get_df_from_protocol_dir(protocol_dir)

Important: check if you need to remove new line and tab or not (depends on dataset).
            you can change it in the init of CatmaDatasetCreation.
            You can validate it using CATMA analyze with some tags to check.


#### extract speakers from txt:

In [ ]:
xml_data = get_single_validated_file_content(
            file_directory_path=os.path.join(protocol_dir, CATMA_XML_ANNOTATION_DIR),
            suffix="xml")
txt_data = get_single_validated_file_content(file_directory_path=protocol_dir,
                                             suffix="txt")
# todo: extract speakers from txt_data
txt_data = txt_data.replace("\n", "  ").replace("\t", " ")
bs_data = BeautifulSoup(xml_data, "xml")
text_catma_id_to_catma_label_id = create_text_catma_id_to_label_mapping(bs_data)
if len(text_catma_id_to_catma_label_id) == 0:
    print("Empty annotation file, skipping it")
    return None
catma_label_id_to_label = create_label_catma_id_to_label_mapping(bs_data)
text_catma_id_to_start_char_end_char = create_text_catma_id_to_start_char_end_char_mapping(bs_data)
not_tagged_start_char_end_char = create_not_tagged_start_char_end_char(bs_data)

catma_ids0 = list(set(text_catma_id_to_catma_label_id.keys()))
catma_ids1 = list(set(text_catma_id_to_start_char_end_char.keys()))
in0not1 = [catma_id for catma_id in catma_ids0 if catma_id not in catma_ids1]
in1not0 = [catma_id for catma_id in catma_ids1 if catma_id not in catma_ids0]
assert len(in0not1) == 0
assert len(in1not0) == 0
if self.remove_new_line_and_tab:
    txt_data = txt_data.replace("\n", "  ").replace("\t",
                                                    " ")
print(
    '''Important: check if you need to remove new line and tab or not (depends on dataset).
    you can change it in the init of CatmaDatasetCreation.
    You can validate it using CATMA analyze with some tags to check.''')
df = pd.DataFrame(text_catma_id_to_catma_label_id.items(),
                  columns=[TEXT_SEGMENT_CATMA_ID_COLUMN, LABEL_CATMA_ID_COLUMN])
df[LABEL_COLUMN] = df.apply(lambda row: catma_label_id_to_label[row[LABEL_CATMA_ID_COLUMN]], axis=1)
df = df.apply(add_start_char_end_char_to_df,
              text_catma_id_to_start_char_end_char=text_catma_id_to_start_char_end_char, axis=1)
not_tagged_df = pd.DataFrame(not_tagged_start_char_end_char, columns=[START_CHAR_COLUMN, END_CHAR_COLUMN])
not_tagged_df[TEXT_SEGMENT_CATMA_ID_COLUMN] = None
not_tagged_df[LABEL_CATMA_ID_COLUMN] = None
not_tagged_df[LABEL_COLUMN] = None
df = pd.concat([df, not_tagged_df], ignore_index=True)
df[FILE_COLUMN] = os.path.basename(protocol_dir)
committee = extract_committee_from_text(txt_data)
df[COMMITTEE_COLUMN] = committee
try:
    protocol_number = extract_protocol_number_from_text(txt_data)
except Exception as exception:
    print(
        f"Using None since couldnt find protocol number for protocol dir: {protocol_dir}, committee is {committee}, exception is: {exception}")
    protocol_number = None
df[PROTOCOL_NUMBER_COLUMN] = protocol_number
df[TEXT_COLUMN] = df.apply(lambda row: " ".join(txt_data[row[START_CHAR_COLUMN]:row[END_CHAR_COLUMN]].split()),
                           axis=1)
# todo: split text in untagged dataset by ['.', ':", "?"]
df[BEFORE_TEXT_CONTEXT_COLUMN] = df.apply(
    lambda row: txt_data[row[START_CHAR_COLUMN] + BEFORE_TEXT_CONTEXT_SIZE:row[
                                                                               END_CHAR_COLUMN] + BEFORE_TEXT_CONTEXT_SIZE],
    axis=1)
df[AFTER_TEXT_CONTEXT_COLUMN] = df.apply(
    lambda row: txt_data[
                row[START_CHAR_COLUMN] + AFTER_TEXT_CONTEXT_SIZE:row[
                                                                     END_CHAR_COLUMN] + AFTER_TEXT_CONTEXT_SIZE],
    axis=1)
df = df.sort_values(by=START_CHAR_COLUMN)
df = df.drop_duplicates(ignore_index=True)